# Golden Adaptive System Comparison

Compare the two systems:
1. **Simple System**: 2 dimensions (fractal + volatility), percentile calibration
2. **Complex System**: 4 dimensions (+ ER + volume), confidence scoring, regime-aware

Run this notebook to decide which system performs better.

In [ ]:
# Setup: Add project root to path
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Fetch Test Data

In [ ]:
import yfinance as yf

def fetch_data(symbol: str, years: int = 2) -> tuple[np.ndarray, list[str]]:
    """Fetch OHLCV data for a symbol."""
    end = datetime.now()
    start = end - timedelta(days=years * 365)
    
    df = yf.download(symbol, start=start, end=end, progress=False)
    
    # Convert to numpy array [open, high, low, close, volume]
    bars = df[['Open', 'High', 'Low', 'Close', 'Volume']].values.astype(np.float64)
    dates = df.index.strftime('%Y-%m-%d').tolist()
    
    print(f"{symbol}: {len(bars)} bars from {dates[0]} to {dates[-1]}")
    return bars, dates

# Fetch data for different asset types
symbols = ['SPY', 'AFRM', 'QQQ', 'GLD']
data = {}

for sym in symbols:
    try:
        bars, dates = fetch_data(sym)
        data[sym] = {'bars': bars, 'dates': dates}
    except Exception as e:
        print(f"Error fetching {sym}: {e}")

## 2. Simple System - Golden EMA (2 Dimensions)

In [ ]:
from backend.computation.golden_ema import (
    calculate_simple_golden_ema,
    golden_ema_indicator,
)

# Test on SPY
spy_bars = data['SPY']['bars']
high, low, close = spy_bars[:, 1], spy_bars[:, 2], spy_bars[:, 3]

# Calculate Simple Golden EMA
simple_indicators = golden_ema_indicator(high, low, close)

print("Simple System (2 dimensions):")
print(f"  Golden EMA range: {np.nanmin(simple_indicators['golden_ema']):.2f} - {np.nanmax(simple_indicators['golden_ema']):.2f}")
print(f"  Alpha range: {np.nanmin(simple_indicators['alpha']):.4f} - {np.nanmax(simple_indicators['alpha']):.4f}")

## 3. Complex System - Golden Adaptive (4 Dimensions)

In [ ]:
from backend.golden_system import (
    compute_golden_adaptive_indicators,
    analyze_dimension_contribution,
    print_dimension_analysis,
)

# Calculate Complex Golden Adaptive indicators
complex_indicators = compute_golden_adaptive_indicators(spy_bars)

# Analyze dimension contributions
analysis = analyze_dimension_contribution(complex_indicators)
print_dimension_analysis(analysis)

## 4. Visual Comparison

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Plot 1: Price with both EMAs
ax1 = axes[0]
ax1.plot(close, label='Close', alpha=0.7, linewidth=1)
ax1.plot(simple_indicators['golden_ema'], label='Simple Golden EMA (2D)', linewidth=2)
ax1.plot(complex_indicators['golden_ema'], label='Complex Golden EMA (4D)', linewidth=2, linestyle='--')
ax1.set_title('SPY: Price vs Golden EMAs')
ax1.legend()
ax1.set_ylabel('Price')

# Plot 2: Alpha comparison
ax2 = axes[1]
ax2.plot(simple_indicators['alpha'], label='Simple Alpha', alpha=0.7)
ax2.plot(complex_indicators['alpha'], label='Complex Alpha', alpha=0.7)
ax2.set_title('Adaptive Alpha Comparison')
ax2.legend()
ax2.set_ylabel('Alpha')

# Plot 3: Confidence (complex only)
ax3 = axes[2]
ax3.fill_between(range(len(complex_indicators['confidence'])), 
                  complex_indicators['confidence'], 
                  alpha=0.5, label='Confidence')
ax3.axhline(y=0.5, color='r', linestyle='--', label='Min Confidence Threshold')
ax3.set_title('Complex System: Confidence Score (Dimension Agreement)')
ax3.legend()
ax3.set_ylabel('Confidence')
ax3.set_xlabel('Bar Index')

plt.tight_layout()
plt.show()

## 5. Backtest Comparison

In [ ]:
from backend.golden_system import backtest_golden_strategy, print_backtest_result

print("=" * 70)
print("BACKTEST: COMPLEX SYSTEM (4 Dimensions)")
print("=" * 70)

results = {}

for sym in data.keys():
    print(f"\n--- {sym} ---")
    result = backtest_golden_strategy(
        data[sym]['bars'], 
        symbol=sym,
        initial_capital=100000,
        min_confidence=0.5,
    )
    results[sym] = result
    print_backtest_result(result)

## 6. Equity Curve Comparison

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (sym, result) in enumerate(results.items()):
    ax = axes[idx]
    ax.plot(result.equity_curve, label=f'{sym} Equity')
    ax.axhline(y=100000, color='gray', linestyle='--', alpha=0.5)
    ax.set_title(f'{sym}: Sharpe={result.sharpe_ratio:.2f}, Return={result.total_return*100:.1f}%')
    ax.set_xlabel('Bar Index')
    ax.set_ylabel('Equity')
    ax.legend()

plt.tight_layout()
plt.show()

## 7. Walk-Forward Validation

In [ ]:
from backend.golden_system import WalkForwardCalibrator

# Run walk-forward on SPY
wf = WalkForwardCalibrator(
    train_bars=252,  # 1 year training
    test_bars=63,    # 3 months testing
    step_bars=21,    # 1 month step
)

print("Running walk-forward calibration on SPY...")
wf_results = wf.run_walk_forward(data['SPY']['bars'], symbol='SPY')
wf.print_walk_forward_summary(wf_results)

## 8. Calibration Comparison

In [ ]:
from backend.calibration import PercentileCalibrator
from backend.golden_system import GoldenCalibrator

# Simple calibrator
simple_cal = PercentileCalibrator()

# Complex calibrator
complex_cal = GoldenCalibrator()

print("Calibration Comparison:")
print("=" * 70)

for sym in ['SPY', 'AFRM']:
    print(f"\n--- {sym} ---")
    
    # Simple
    simple_params = simple_cal.calibrate(sym, data[sym]['bars'], data[sym]['dates'])
    print(f"Simple System:")
    print(f"  RSI Oversold: {simple_params.rsi_oversold:.1f} (vs fixed 30)")
    print(f"  RSI Overbought: {simple_params.rsi_overbought:.1f} (vs fixed 70)")
    print(f"  ER Trending: {simple_params.er_trending:.3f} (vs fixed 0.6)")
    
    # Complex
    complex_params = complex_cal.calibrate(data[sym]['bars'], sym)
    print(f"\nComplex System:")
    print(f"  Min Confidence: {complex_params.min_confidence:.3f}")
    print(f"  Weights: fractal={complex_params.w_fractal:.2f}, ER={complex_params.w_efficiency:.2f}, vol={complex_params.w_volatility:.2f}, volume={complex_params.w_volume:.2f}")
    print(f"  ATR Stops: trend={complex_params.atr_stop_trending:.1f}x, MR={complex_params.atr_stop_mr:.1f}x")
    print(f"  Regime %: trend={complex_params.regime_pct_trending*100:.1f}%, MR={complex_params.regime_pct_mr*100:.1f}%, neutral={complex_params.regime_pct_neutral*100:.1f}%")

## 9. Summary Table

In [ ]:
import pandas as pd

summary_data = []
for sym, result in results.items():
    summary_data.append({
        'Symbol': sym,
        'Total Return %': f"{result.total_return * 100:.1f}%",
        'Sharpe Ratio': f"{result.sharpe_ratio:.2f}",
        'Max Drawdown %': f"{result.max_drawdown * 100:.1f}%",
        'Win Rate %': f"{result.win_rate * 100:.1f}%",
        'Trades': result.n_trades,
        'Avg Confidence': f"{result.avg_confidence:.2f}",
    })

df = pd.DataFrame(summary_data)
print("\nComplex System (4D) Backtest Summary:")
print(df.to_string(index=False))

## 10. Conclusion

Based on the backtests above, decide which system to use:

- **If Complex system has better Sharpe and lower drawdown**: Keep both, use Complex for live trading
- **If Simple system performs similarly**: Delete `backend/golden_system/` folder (simpler is better)
- **If both underperform**: Parameters need more tuning or strategy logic needs adjustment